In [0]:

/* 
=====================================================
Building Exploratory Data Analysis on the Gold Layer:
=====================================================
Purpose: 
  - Build Exploratory Data Analysis on the Gold Layer
  - Build Cumlative values for sum of sales_amount over order_date column

*/


-- EXERCISE 1: Build Cumlative values for sum of sales_amount over order_date column
select 
  order_date,
  sum(sum(sales_amount)) over (order by order_date rows between 1 preceding and current row) as cumulative_sales_amount,
  sum(sum(sales_amount)) over (order by order_date) as daily_sales
from datawarehouse.gold.fact_sales
where order_date is not null
group by order_date --, year(order_date), month(order_date)
order by 1;

-- EXERCISE 2: Build Cumlative values for sum of sales_amount over order_date column
select
  order_date,
  total_sales,
  avg_price,
  cumulative_sales_by_1_row,
  cumulative_sales,
  round(moving_average_price, 2) as moving_average_price
from (
  select 
    order_date,
    total_sales,
    avg_price,
    sum(total_sales) over (order by order_date rows between 1 preceding and current row) as cumulative_sales_by_1_row,
    sum(total_sales) over (order by order_date) as cumulative_sales,
    avg(avg_price) over (order by order_date) as moving_average_price
  from (
    SELECT 
      order_date,
      sum(sales_amount) as total_sales,
      avg(price) as avg_price
    from datawarehouse.gold.fact_sales
    where order_date is not null
    group by order_date
  )
);



select 
order_date,
sum(sales_amount) as total_sales,
sum(sales_amount) over (order by order_date) as cumulative_sales
from datawarehouse.gold.fact_sales
where order_date is not null
group by order_date
order by order_date;





-- compare product performance for each year 
with yearly_product_sales(
  select 
    year(order_date) as order_year,
    d.prodcut_name,
    sum(sales_amount) as current_sales
  from datawarehouse.gold.fact_sales f
  left join datawarehouse.gold.dim_prodcut d
    on f.product_key = d.product_key
  where order_date is not null
  group by d.prodcut_name, year(order_date)
)
select
order_year,
prodcut_name,
current_sales,
avg(current_sales) over (partition by prodcut_name) as avg_sales,
current_sales - avg_sales as sales_performance,
case
  when sales_performance > 0 then 'Above Average'
  when sales_performance = 0 then 'Average'
  else 'Below Average'
end avg_flag,
current_sales - lag(current_sales) over (partition by prodcut_name order by order_year) as sales_change_by_year

-- lag(current_sales) over (order by order_year) as lag_sales,
from yearly_product_sales
order by prodcut_name, order_year;




-- ===================================
-- Which categories contribute the most to overall sales
with product_sales as 
(
  select 
    d.category as category,
    sum(f.sales_amount) as sales_by_category
    -- sum(sum(f.sales_amount)) over (partition by d.category) / sum(f.sales_amount) * 100 as sales_percentage
  from datawarehouse.gold.fact_sales f
  left join datawarehouse.gold.dim_prodcut d
    on f.product_key = d.product_key
  where order_date is not null 
  group by d.category
)
select
  category,
  sales_by_category,
  round(sales_by_category / sum(sales_by_category) over () * 100, 2) as sales_percentage
from product_sales;













Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.